# XGBoost Regressor

---

**Источники:**

[XGBoost](https://neerc.ifmo.ru/wiki/index.php?title=XGBoost)

[Виды ансамблей](https://neerc.ifmo.ru/wiki/index.php?title=%D0%92%D0%B8%D0%B4%D1%8B_%D0%B0%D0%BD%D1%81%D0%B0%D0%BC%D0%B1%D0%BB%D0%B5%D0%B9)

[XGBoost Parameters](https://xgboost.readthedocs.io/en/latest/parameter.html)

[Эвристический алгоритм](https://ru.wikipedia.org/wiki/%D0%AD%D0%B2%D1%80%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC#:~:text=%D0%AD%D0%B2%D1%80%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9%20%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%20(%D1%8D%D0%B2%D1%80%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B0)%20%E2%80%94%20%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC,%D1%80%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%BD%D0%B5%20%D0%BC%D0%BE%D0%B6%D0%B5%D1%82%20%D0%B1%D1%8B%D1%82%D1%8C%20%D0%BD%D0%B0%D0%B9%D0%B4%D0%B5%D0%BD%D0%BE.)

[Настройка параметров XGBoost](https://russianblogs.com/article/2308221344)

[Python API Reference](https://xgboost.readthedocs.io/en/latest/python/python_api.html)

[Chapter 12 Gradient Boosting](https://bradleyboehmke.github.io/HOML/gbm.html)

---


## Подготовка окружения

In [ ]:
# ВНИМАНИЕ: необходимо удостовериться, что виртуальная среда выбрана правильно!

# Для MacOS/Ubuntu
# !which pip

# Для Windows
# !where pip

In [ ]:
# !conda install matplotlib numpy scikit-learn seaborn -y

In [ ]:
import numpy as np

np.__version__

In [ ]:
import pandas as pd

pd.__version__

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.__version__

In [ ]:
import seaborn as sns

sns.__version__

## Загрузка данных

[Источник (FuelConsumption)](https://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64)

In [ ]:
df = pd.read_csv("./../../data/FuelConsumptionCo2.csv")
df

## Разделение данных

In [ ]:
from sklearn.model_selection import train_test_split

y = df['CO2EMISSIONS'].copy()
X = df[['ENGINESIZE']].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

**Одна из самых популярных и эффективных реализаций алгоритма градиентного бустинга на деревьях на 2019-й год.**


В основе XGBoost лежит алгоритм градиентного бустинга деревьев решений. 

**Градиентный бустинг** — это техника машинного обучения для задач классификации и регрессии, которая строит модель предсказания в форме ансамбля слабых предсказывающих моделей, обычно деревьев решений. Обучение ансамбля проводится последовательно в отличие, например от бэггинга. 

**Ансамбль алгоритмов (методов)** — метод, который использует несколько обучающих алгоритмов с целью получения лучшей эффективности прогнозирования, чем можно было бы получить от каждого обучающего алгоритма по отдельности.

На каждой итерации вычисляются отклонения предсказаний уже обученного ансамбля на обучающей выборке. Следующая модель, которая будет добавлена в ансамбль будет предсказывать эти отклонения. Таким образом, добавив предсказания нового дерева к предсказаниям обученного ансамбля мы можем уменьшить среднее отклонение модели, котрое является таргетом оптимизационной задачи. Новые деревья добавляются в ансамбль до тех пор, пока ошибка уменьшается, либо пока не выполняется одно из правил "ранней остановки".


XGBoost поддерживает все возможности таких библиотек как scikit-learn с возможностью добавлять регуляризацию. Поддержаны три главные формы градиетного бустинга:

- Стандартный градиентный бустинг с возможностью изменения скорости обучения(learning rate).
- Стохастический градиентный бустинг с возможностью семплирования по строкам и колонкам датасета.
- Регуляризованный градиентный бустинг с L1 и L2 регуляризацией.

## Обучение модели (Train)

In [3]:
# разделить независимую и зависимую переменные / train и test

from sklearn.model_selection import train_test_split

y = df['CO2EMISSIONS'].copy()
X = df.drop(['CO2EMISSIONS', 'MAKE', 'MODEL', 'VEHICLECLASS', 'TRANSMISSION', 'FUELTYPE'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### `class xgboost.XGBRegressor(**kwargs)`

**[Остальные параметры см. 7_2_xgboost_params](./7_2_xgboost_params.ipynb)**

- **`n_estimators (int)`**
    - Количество деревьев с градиентным усилением. Эквивалентно количеству раундов boost-инга (boosting rounds).

- **`max_depth (int) [default=6]`**
    - Максимальная глубина дерева. Увеличение этого значения сделает модель более сложной и с большей вероятностью переобучится (overfit).
    - Допустимые значения: $[0, \infty]$, `0` принимается только в lossguided для роста, когда tree_method установлен как hist и указывает отсутствие ограничения на глубину. Помните, что XGBoost агрессивно расходует память при обучении глубокого дерева.

- **`learning_rate (float) [default=0.3, alias: eta]`**
    - Уменьшение размера шага, используемое в обновлении, чтобы **предотвратить переобучение (overfitting)**. После каждого шага повышения мы можем напрямую получить веса новых функций, а eta уменьшает веса функций, чтобы сделать процесс повышения более консервативным.
    - Допустимые значения: $[0, 1]$

In [4]:
# импортировать пакет xgboost
import xgboost as xg

# создать объект XGBRegressor
xgb_r = xg.XGBRegressor(n_estimators = 10, max_depth=6)

ModuleNotFoundError: No module named 'xgboost'

### `fit(X, y, *, sample_weight=None, base_margin=None, eval_set=None, eval_metric=None, early_stopping_rounds=None, verbose=True, xgb_model=None, sample_weight_eval_set=None, feature_weights=None, callbacks=None)`

**[Остальные параметры см. 7_2_xgboost_params](./7_2_xgboost_params.ipynb)**

- **`X (array_like)`** - Матрица признаков.
    
    
- **`y (array_like)`** - Labels ("правильные ответы").


- **`early_stopping_rounds (int)`** 
    - Активирует раннюю остановку.
    - Чтобы продолжить обучение, показатель валидации должен улучшаться хотя бы раз в каждом раунде `early_stopping_rounds`.
    - Требуется хотя бы один элемент в `eval_set`.
    - Метод возвращает модель с последней итерации (а не самой лучшей).
    - Если в `eval_set` более одного элемента, для ранней остановки будет использоваться *последняя запись* (last entry).
    - Если в `eval_metric` более одной метрики, для ранней остановки будет использоваться *последняя метрика* (last metric).
    - Если произойдет ранняя остановка, модель будет иметь три дополнительных поля: `clf.best_score`, `clf.best_iteration` и `clf.best_ntree_limit`.

In [ ]:
# обучить модель
xgb_r.fit(X_train, y_train)

## Оценка качества модели (Evaluation)

#### `predict(data, output_margin=False, ntree_limit=None, validate_features=True, base_margin=None)`

**[Остальные параметры см. 7_2_xgboost_params](./7_2_xgboost_params.ipynb)**

- **`data (numpy.array/scipy.sparse)`**
    - Данные для прогнозирования.
    
    
- **`validate_features (bool)`**
    - Если это `True`, то будет произведена проверка (validate), что `feature_names` для `Booster` и `data` идентичны. В противном случае предполагается, что `feature_names` совпадают.

In [ ]:
# использовать обученную модель для предсказания на test выборке
y_predicted = xgb_r.predict(X_test)

# вывести результаты предсказания
print(f'Variance score: {xgb_r.score(X_test, y_test)}')    # Coefficient of determination R^2 of the prediction
print(f"Residual sum of squares: {np.mean((y_predicted - y_test) ** 2)}")    # MSE

In [ ]:
from sklearn.metrics import r2_score

r2_score(y_true=y_test, y_pred=y_predicted)    # эквивалентно simple_regr.score(X_test, y_test)

In [ ]:
from sklearn.metrics import mean_squared_error

# MSE
mean_squared_error(y_test, y_predicted, squared=True)   # эквивалентно np.mean((y_predicted - y_test) ** 2)